In [3]:
import json
import psycopg2
import pandas as pd

import requests
import urllib
import gzip

In [4]:
# загрузка gz-архива по ссылке
def load_from_url(url):
    fname = url[url.rfind('/')+1:].strip()
    gz = urllib.request.urlopen(url).read()
    
    f = open(f'data/{fname}', "wb")
    f.write(gz)
    f.close()
    
    return fname



In [15]:
# парсинг csv одного курса из архива в DataFrame
def parse_structure():
    df['course_id'] = df.apply(lambda x: x[1][:x[1].find('+type@chapter')].replace('block-v1:', ''), axis=1)
    df['chapter_id'] = df.apply(lambda x: x[1].replace(f'block-v1:{x["course_id"]}+type@chapter+block@', ''), axis=1) 
    df['sequential_id'] = df.apply(lambda x: x[2].replace(f'block-v1:{x["course_id"]}+type@sequential+block@', ''), axis=1)
    df['vertical_id'] = df.apply(lambda x: x[3].replace(f'block-v1:{x["course_id"]}+type@vertical+block@', ''), axis=1)
    
    df['item_type'] = df.apply(lambda x: x[4].replace(f'block-v1:{x["course_id"]}+type@', ''), axis=1)
    df['item_type'] = df.apply(lambda x: x['item_type'][:x['item_type'].find('+')], axis=1)
                                                      
    df['item_id'] = df.apply(lambda x: x[4].replace(f'block-v1:{x["course_id"]}+type@{x["item_type"]}+block@', ''), axis=1)
                                                    
    df[['chapter_name', 'sequential_name', 'vertical_name', 'item_name']] = df[[5, 6, 7, 8]]
                                                          

### Функции-обертки для обращений к БД

In [6]:
def find_course(course_extid):
    cursor.callproc("openedu_structure.find_course", [course_extid,])
    return cursor.fetchone()[0]

In [7]:
def add_chapter(course_id, chapter_extid, chapter_name):
    cursor.callproc("openedu_structure.add_chapter", [course_id, chapter_extid, chapter_name,])
    return cursor.fetchone()[0]

In [8]:
def add_sequential(chapter_id, sequential_extid, sequential_name):
    cursor.callproc("openedu_structure.add_sequential", [chapter_id, sequential_extid, sequential_name,])
    return cursor.fetchone()[0]

In [9]:
def add_vertical(sequential_id, vertical_extid, vertical_name):
    cursor.callproc("openedu_structure.add_vertical", [sequential_id, vertical_extid, vertical_name,])
    return cursor.fetchone()[0]

In [10]:
def add_item(vertical_id, item_extid, item_type, item_name):
    cursor.callproc("openedu_structure.add_item", [vertical_id, item_extid, item_type, item_name,])
    return cursor.fetchone()[0]

### Загрузка курса в БД

In [11]:
def load_course_to_db():
    conn = psycopg2.connect(dbname='test', user='postgres', 
                        password='****', host='VM-****')
    cursor = conn.cursor()

    course_extid = 0
    chapter_extid = 0
    sequential_extid = 0
    vertical_extid = 0
    item_extid = 0

    for row in df.iterrows():
        if course_extid != row[1]['course_id']:
            course_extid = row[1]['course_id']
            course_id = find_course(course_extid)
            if not course_id:
                continue

        if chapter_extid != row[1]['chapter_id']:
            chapter_extid = row[1]['chapter_id']
            chapter_id = add_chapter(course_id, chapter_extid, row[1]['chapter_name'])

        if sequential_extid != row[1]['sequential_id']:
            sequential_extid = row[1]['sequential_id']
            sequential_id = add_sequential(chapter_id, sequential_extid, row[1]['sequential_name'])

        if vertical_extid != row[1]['vertical_id']:
            vertical_extid = row[1]['vertical_id']
            vertical_id = add_vertical(sequential_id, vertical_extid, row[1]['vertical_name'])

        if item_extid != row[1]['item_id']:
            item_extid = row[1]['item_id']
            #print(vertical_id, item_extid, row[1]['item_type'], row[1]['item_name']) 
            add_item(vertical_id, item_extid, row[1]['item_type'], row[1]['item_name'])    

    
    conn.commit()
    cursor.close()
    return course_id, course_extid

In [12]:
# подключение к БД
conn = psycopg2.connect(dbname='test', user='postgres', 
                        password='pg215', host='VM-AS494')
cursor = conn.cursor()

In [13]:
with open('qq.txt', 'r', encoding='utf-8') as g:
    for line in g:
        print(f'Обрабатывается {line}')
        fname = load_from_url(line)
        
        with gzip.open(fname) as f:
            df = pd.read_csv(f, sep=';', header=None)
        
        parse_structure()
        print(f'Размер датафрейма к загрузке {df.shape}')
        
        print(f'Загружен в БД контент для курса  {load_course_to_db()}')


https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_ECONOM_spring_2019.csv.gz

(120, 19)
(133, 'hse+ECONOM+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_EXACTS_spring_2019.csv.gz

(209, 19)
(159, 'hse+EXACTS+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FIL_spring_2019.csv.gz

(274, 19)
(173, 'hse+FIL+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FORLANG_spring_2019.csv.gz

(226, 19)
(227, 'hse+FORLANG+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FUNDLAW_spring_2019.csv.gz

(97, 19)
(234, 'hse+FUNDLAW+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_LABLAW_spring_2019.csv.gz

(219, 19)
(305, 'hse+LABLAW+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_MACROEC_spring_2019.csv.gz

(139, 19)
(335, 'hse+MACROEC+spring_2019')
https://207918.selcdn.ru/jCJ

In [14]:
conn.commit()
cursor.close()

In [7]:
features = ['course_id', 'chapter_id', 'chapter_name', 'sequential_id', 'sequential_name', 
            'vertical_id', 'vertical_name', 'item_type', 'item_id',  'item_name']
df[features].head()

,course_id,chapter_id,chapter_name,sequential_id,sequential_name,vertical_id,vertical_name,item_type,item_id,item_name
0,hse+DATPRO+spring_2019,8411723edf2f4d8cb6b130fb8023a26d,О Вышке,b74b1284929046d2b280f3c560f0e896,Промо-ролик,e444e0cf02854ed9b516481d39f9a342,Промо-ролик,video,5b50ca94e2774d6f83da03543e39bf50,Видео о ВШЭ
1,hse+DATPRO+spring_2019,911bd2c030644cc9841a2950b62489c0,Тема 1. Основы информационной безопасности. Ос...,da6736e32aae4ac6922f6c55129d312f,Лекция 1,f27a9e3ace954d9a9838ee229f116f94,Понятие информации,video,261019e7ca7d40cabf469069dbef052c,Понятие информации
2,hse+DATPRO+spring_2019,911bd2c030644cc9841a2950b62489c0,Тема 1. Основы информационной безопасности. Ос...,da6736e32aae4ac6922f6c55129d312f,Лекция 1,dd37a044619949568f9ae7323be7850d,Доступ к информации,video,d1efc98f8244445f9d011479366f9fb2,Доступ к информации
3,hse+DATPRO+spring_2019,911bd2c030644cc9841a2950b62489c0,Тема 1. Основы информационной безопасности. Ос...,da6736e32aae4ac6922f6c55129d312f,Лекция 1,f6b6aa6909cd40abbc74e2fd1845d549,Информационные системы,video,19a06eb43b0f461e88ac12340ed9cb54,Информационные системы
4,hse+DATPRO+spring_2019,911bd2c030644cc9841a2950b62489c0,Тема 1. Основы информационной безопасности. Ос...,da6736e32aae4ac6922f6c55129d312f,Лекция 1,f915b0bf93c24c859d741e11993375f8,Обработка информации,video,94ef7b162a3442a19eb4c3dd436afc45,Обработка информации


In [12]:
conn = psycopg2.connect(dbname='test', user='postgres', 
                        password='pg215', host='VM-AS494')
cursor = conn.cursor()

In [27]:
copy_sql = """
  copy open_edu_course (id, name, ext_id)
  from stdin with
    csv
    header
    delimiter as ';'
"""
from_csv = 'course_for_load1.csv'
with open(from_csv, 'r') as f:
  cursor.copy_expert(sql=copy_sql, file=f)
  conn.commit()
  cursor.close()

In [18]:
?cursor.copy_expert

In [11]:
conn.commit() 

In [3]:
cursor.execute("SELECT * FROM openedu_structure.course where ext_id like '%FUNDLAW%'")

In [4]:
for row in cursor:
    print(row)

(230, '(DEMO)Основы права', 'hse+FUNDLAW+DEMO', None)
(231, '(2018-2)Основы права', 'hse+FUNDLAW+fall_2018', None)
(232, 'Основы права', 'hse+FUNDLAW+fall_2019', None)
(233, 'Основы права', 'hse+FUNDLAW+fall_2019_dvfu', None)
(234, '(2019-1)Основы права', 'hse+FUNDLAW+spring_2019', None)
(235, 'Основы права', 'hse+FUNDLAW+spring_2020', None)
(236, '(2018-2)Основы права', 'hse+FUNDLAW+stud_fall_2018', None)
(237, 'Основы права', 'hse+FUNDLAW+stud_fall_2019', None)
(238, 'Основы права', 'hse+FUNDLAW+workshop_adaptive', None)


In [28]:
from psycopg2 import sql

In [29]:
columns = ('country_name_ru', 'airport_name_ru', 'city_code')
stmt = sql.SQL('SELECT {} FROM {} LIMIT 5').format(
            sql.SQL(',').join(map(sql.Identifier, columns)),
            sql.Identifier('airport')
        )

In [30]:
values = [
        (3, 'однажды в ... ;;;'),
        (4, 'хэлло, товарищи!'),
    ]

In [31]:
insert = sql.SQL('INSERT INTO test_table (id, name) VALUES {}').format(
        sql.SQL(',').join(map(sql.Literal, values))
    )
    

In [32]:
cursor.execute(insert)

UniqueViolation: ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "test_table_pkey"
DETAIL:  Ключ "(id)=(3)" уже существует.


In [22]:
insert

Composed([SQL('INSERT INTO test_table (id, name) VALUES '), Composed([Literal((3, 'однажды в ... ;;;')), SQL(','), Literal((4, 'хэлло, товарищи!'))])])

In [36]:
conn.commit()  # или надо использовать conn.autocommit = True

### На память: для перезагрузки конфига в  psql запустить select pg_reload_conf()

## Скачивание по массиву ссылок

https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_ECONOM_spring_2019.csv.gz

structure_hse_ECONOM_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_EXACTS_spring_2019.csv.gz

structure_hse_EXACTS_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FIL_spring_2019.csv.gz

structure_hse_FIL_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FORLANG_spring_2019.csv.gz

structure_hse_FORLANG_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FUNDLAW_spring_2019.csv.gz

structure_hse_FUNDLAW_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_LABLAW_spring_2019.csv.gz

structure_hse_LABLAW_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_MACROEC_spring_2019.csv.gz

structure_hse_MACROEC_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW

In [79]:
import gzip

,0,1,2,3,4,5,6,7,8
0,0,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,О Вышке,Промо-ролик,Промо-ролик,Промо-ролик
1,1,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 1,Предмет экономической психологии
2,2,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 2,Предмет экономической психологии
3,3,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,История экономической психологии,История экономической психологии
4,4,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Методы экономической психологии,Методы экономической психологии


In [84]:
parse_structure()

In [85]:
df

,0,1,2,3,4,5,6,7,8,course_id,chapter_id,sequential_id,vertical_id,item_type,item_id,chapter_name,sequential_name,vertical_name,item_name
0,0,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,О Вышке,Промо-ролик,Промо-ролик,Промо-ролик,hse+ECPSY+spring_2019,6c1d114713584f3f90f15200911361f5,6bd9c4040c384db0adc044d826f67b48,b20a29b9ace24655ac0e649d07ef8a88,video,0079332f8dba44ee8dc62bdb87e36acf,О Вышке,Промо-ролик,Промо-ролик,Промо-ролик
1,1,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 1,Предмет экономической психологии,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,e84e2770efd04b09ad487024427b5ea3,d828db02387a41eebb6f0c3ab46f736b,video,08305cf9df4b4fd5ad7fe712117c800d,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 1,Предмет экономической психологии
2,2,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 2,Предмет экономической психологии,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,e84e2770efd04b09ad487024427b5ea3,c283dadf180643899cbe5de78efdd77b,video,8eba95675ac7414fa0e965031ab07dee,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 2,Предмет экономической психологии
3,3,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,История экономической психологии,История экономической психологии,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,e84e2770efd04b09ad487024427b5ea3,94db82d471a94052a16038175c861d22,video,0bf8b4fc19df471fb9866da6f52ea426,Неделя 1: Экономическая психология,Лекция 1,История экономической психологии,История экономической психологии
4,4,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Методы экономической психологии,Методы экономической психологии,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,e84e2770efd04b09ad487024427b5ea3,bb5df3a8a76a479aaf1dccd594f0d566,video,4e2716bda32f4d899230f5ac17541dbc,Неделя 1: Экономическая психология,Лекция 1,Методы экономической психологии,Методы экономической психологии
5,5,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,О курсе,О курсе,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,e84e2770efd04b09ad487024427b5ea3,f962fabc68d94cefb5ab4996228a174c,video,0c823967d4874b22b74cd0bd5d0f880e,Неделя 1: Экономическая психология,Лекция 1,О курсе,О курсе
6,6,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@problem+bl...,Неделя 1: Экономическая психология,Тест 1,Тест 1,Тест к лекции №1,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,9838dc78885c490687b6dd6e2fce43a2,f63ac0e3aac943bb9f2bd5c5ce15ddfe,problem,f06d73e6a1e341119feb11d1fca76791,Неделя 1: Экономическая психология,Тест 1,Тест 1,Тест к лекции №1
7,7,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v